In [581]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import re
from datetime import date

In [582]:
import os
os.chdir("C:\\Users\\newton.sathyavety\\OneDrive\\Documents\\Vomanic")

In [630]:
file_1 = "flip_oc_0.json"
file_2 = "meat_0.json"

with open(file_1,"r+") as f:
    data_1 = json.load(f)

In [631]:
len(data_1)

2000

In [632]:
i = 0
lst = []
for index,each_mail in enumerate(data_1):
    # if index == 1000:
    #     break
    # print(each_mail["_source"]["rawHtml"])
    soup = bs(each_mail["_source"]["rawHtml"], "html.parser")
    # print(soup.prettify())
    web_title = soup.find("title")
    web_body = soup.find("body")

    # print(len(web_body.select("td.container-padding")))
    final_output = {}

    if len(web_body.select("td.container-padding")) > 2:
        output = {}

        ##########  ORDER DETAILS  #####
        orderData = []
        orderDelivery= None
        orderAmount = None
        orderDeliveryAddress = None
        orderDeliveryAmount = 0

        orderDetails = web_body.select("td.container-padding")[0].find_all("table",attrs={"class":"force-row"})
        if orderDetails:
            orderData.append(orderDetails[0].text)
            orderData.append(orderDetails[1].text)
        orderDetailsData = web_body.select("td.container-padding")[2].find("td",attrs={"class":"col"}).select("span:nth-child(2n)")
        if orderDetailsData:
            orderDelivery = orderDetailsData[0].text
            orderAmount = re.findall("\d+",orderDetailsData[1].text.strip().replace("[#@$&!]","").replace("+",""))[-1]
            # print(orderAmount)
        # print(orderData)
        # print(orderDelivery,orderAmount)

        ######### ORDER DELIVERY ADDRESS ###########
        orderDeliveryAddressData = web_body.select("td.container-padding")[2].find_all("table",attrs={"class":"force-row"})
        if orderDeliveryAddressData:
            orderDeliveryAddress = orderDeliveryAddressData[1].find("span").text
        # print(orderDeliveryAddress)

        #### ORDER COST #########
        item_price = None
        deliver_price = None
        total_price = None
        order_cost = web_body.select("table.container")[-2].select("p span")
        if len(order_cost) // 2 == 3:
            item_price = order_cost[1].text
            deliver_price = order_cost[3].text
            total_price = order_cost[5].text
        if len(order_cost) // 2 == 2:
            item_price = order_cost[1].text
            total_price = order_cost[3].text
        # print(item_price,deliver_price,total_price)

        output["name"] = orderData[0].split(",")[0].strip().split("Hi".capitalize())[-1].strip()
        output["order_date"] = orderData[1].split(",")[0].split("Order placed on")[-1].strip()+" "+orderData[1].split(",")[1].strip()[:4]
        output["order_id"] = orderData[1].split(",")[1].strip().split("Order ID")[-1]
        output["order_delivery_address"] = orderDeliveryAddress.replace("\n","").replace("\r","").replace(",","").replace('[#@$&!]',"")
        output["order_delivery_amount"] = str(orderDeliveryAmount).strip()
        output["order_amount"] = orderAmount.strip()

        # print(orderData[1].split(",")[1])
        # print(output)
    else:
        output = {}
        order_tables = soup.select("table.force-row")
        product_table = soup.select("table.container")

        orderID = None
        orderPlacedName = None
        orderPlaced = None
        orderDeliveryAddress = None
        orderDeliveryAmount = 0

        order = []
        for i in range(len(order_tables)):
            # print(order_tables[i].prettify())
            order_table_details = order_tables[i].find("td",attrs={"class":"col"})
            # print(order_table_details.find_all("p"))
            if order_table_details:
                # print(order_tables[i].find("td",attrs={"class":"col"}).find_all("p"))
                order_table_details_p = order_table_details.find_all("p")
                order_placed_name_data = order_table_details_p[0]
                order_placed_data = order_table_details_p[1]
                # order_view_link = order_table_details.find_all("p")[2]
                for record in order_placed_name_data.find_all("span"):
                    if record:
                        order.append(record.text)

                if order_placed_data.select("p > span"):
                    orderID = order_placed_data.select("p > span")[0].text

                # print(order_placed_data)
                if order_placed_data.select('p > a'):
                    order_link = order_placed_data.select('p > a')[0]["href"]
                    # print(order_link)
                # print(order_placed_details)


                # order.append(order_details)

        orderPlacedName = order[0].strip().split("Hi")[-1].strip()[:-1].strip()
        orderPlaced = order[1].strip().split("Ordered placed on")[-1].strip()
        # print(order_placed_name,order_placed,order_id,order_link)

        product = []
        product_value = []
        for i in range(len(product_table)):
            # if product_table[i].find("td",attrs={"class":"link"}):
            #     gift_voucher = product_table[i].find("td",attrs={"class":"link"}).text

            # print(product_table[0].prettify())
            table = product_table[i].select('table.table-striped')
            # print(table)
            if table:
                # table_df = pd.read_html(str(table))[0]
                # print(table_df)
                gift_voucher_header = table[0].find('thead').find_all('th')
                for header in gift_voucher_header:
                    product.append(header.text)
                    # print(header.text)
                gift_voucher_values = table[0].find("tbody").select("tr td")
                for rng in range(2,len(gift_voucher_values)):
                    product_value.append(gift_voucher_values[rng].text)
                    # print(gift_voucher_values[rng].text)

        order_detail_data = {}
        if product and product_value:
            for idx in range(len(product)):
               order_detail_data[product[idx].strip().lower()] = product_value[idx].strip()

        output["name"] = orderPlacedName.strip()
        output["order_date"] = orderPlaced.strip()
        output["order_id"] = orderID.strip()
        output["order_delivery_address"] = str(orderDeliveryAddress).replace('[#@$&!]',"")
        output["order_delivery_amount"] = str(orderDeliveryAmount).strip()
        output["order_amount"] = re.findall("\d+",order_detail_data["amount"])[-1]

        # print(output)

    final_output = output
    final_output["company_name"] = each_mail["_source"]["companyName"]
    final_output["mail"] = each_mail["_source"]["to"]
    final_output["received_date"] = each_mail["_source"]["emailTimeStamp"]
    # final_output["received_date"] = date.fromtimestamp(each_mail["_source"]["emailTimeStamp"])

    print(final_output)
    lst.append(final_output)
    print("*"*100)


{'name': 'Manashi Saha', 'order_date': 'Feb 21, 2021', 'order_id': 'OD121076230805862000', 'order_delivery_address': 'None', 'order_delivery_amount': '0', 'order_amount': '25', 'company_name': 'flipkart', 'mail': 'manashisaha12333@gmail.com', 'received_date': 1613907461000}
****************************************************************************************************
{'name': 'Nigaran Gs', 'order_date': 'Feb 08, 2021', 'order_id': 'OD120957862971179000', 'order_delivery_address': 'None', 'order_delivery_amount': '0', 'order_amount': '300', 'company_name': 'flipkart', 'mail': 'nigaran771@gmail.com', 'received_date': 1612723401000}
****************************************************************************************************
{'name': 'Harshal Mevada', 'order_date': 'Feb 03, 2021', 'order_id': 'OD120920728474237000', 'order_delivery_address': 'None', 'order_delivery_amount': '0', 'order_amount': '25', 'company_name': 'flipkart', 'mail': 'naynamevada2@gmail.com', 'received_date

In [633]:
df = pd.DataFrame(lst,columns=["name", "order_date","order_id","order_amount","order_delivery_address","company_name","mail","received_date"])

In [634]:
df.to_csv("parseddata.csv",index=False)

In [635]:
import os
os.getcwd()

'C:\\Users\\newton.sathyavety\\OneDrive\\Documents\\Vomanic'